## Imports

In [2]:
%load_ext autoreload
%autoreload

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics
import pandas as pd
import numpy as np
from torch_geometric.data import Data
import pandapower as pp
import networkx as nx
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

from torch.utils.tensorboard import SummaryWriter
from arquitecturas import GNN_global, FCNN_global
from metric import NormalizedError


/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/iboero/miniconda3/envs/proy/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/tmp/ipykernel_29402/2187797291.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  im

In [3]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

red = "118" # o 118

## Levantar Topologia de Red

In [4]:
# Levantamos la red de IEEE118

if red=="30":
    net = pp.networks.case30()
    z_trafos = net.trafo[['hv_bus', 'lv_bus']].to_numpy().astype(np.int32)
else:
    net = pp.networks.case118()
    z_trafos =  np.array([
        [0, 0.0267],
        [0, 0.0382],
        [0, 0.0388],
        [0, 0.0375],
        [0, 0.0386],
        [0, 0.0268],
        [0, 0.0370],
        [0.0013, 0.016],
        [0, 0.0370],
        [0.0017, 0.0202],
        [0, 0.0370],
        [0.0282, 0.2074],
        [0.0003, 0.00405]
    ])

# Armar matirz de adyacencia
lineas = net.line[['from_bus', 'to_bus']].to_numpy().astype(np.int32)
trafos = net.trafo[['hv_bus', 'lv_bus']].to_numpy().astype(np.int32)
edge_index = np.append(lineas, trafos,axis=0)
edge_index = torch.Tensor(edge_index).t().type(torch.int64).to(device)


# Armar matriz de pesos
k = 10
z_lineas =  net.line[['r_ohm_per_km', 'x_ohm_per_km']].to_numpy() * 100 / np.expand_dims((net.bus['vn_kv'].to_numpy()[net.line['from_bus'].to_numpy()])**2,axis=1)
edge_weights = np.append(z_lineas, z_trafos,axis=0)
edge_weights = torch.Tensor(np.e**(-k*(edge_weights[:,0]**2 + edge_weights[:,1]**2))).to(device)

# Armar mascara de generadores para la salida
idx_gen = net.gen["bus"].to_numpy()
idx_grid = net.ext_grid["bus"].to_numpy()
idx_gens = np.append(idx_gen,idx_grid,axis=0)
feature_mask = np.zeros(len(net.bus.index), dtype=int)
feature_mask[idx_gens] = 1
feature_mask = torch.Tensor(feature_mask).type(torch.int32).to(device)

/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/home/iboero/.local/li

## Levanatar Datos

In [5]:
# Levantar los datos
datos_path = './data'

if red == "30":
    X = np.load(datos_path+'/input30.npy')
    y = np.load(datos_path+'/p_opt30.npy')
else:
    X = np.load(datos_path+'/input118.npy')
    y = np.load(datos_path+'/p_opt118.npy')

# Separar en X e Y 
X_tensor = torch.Tensor(X).to(device)
y_tensor_global = torch.Tensor(y[:,:,0]).to(device)[:, feature_mask == 1]


In [6]:
# Generar loaders para el entrenamiento
batch_size = 128

dataset_global = TensorDataset(X_tensor, y_tensor_global)
data_train_global ,data_test_global  = train_test_split(dataset_global ,test_size=0.2,random_state=42)
data_train_global ,data_val_global  = train_test_split(data_train_global ,test_size=0.1,random_state=42)

train_loader_global  = DataLoader(data_train_global , batch_size=batch_size)
val_loader_global  = DataLoader(data_val_global , batch_size=batch_size)
test_loader_global  = DataLoader(data_test_global , batch_size=batch_size)

## Entrenamiento


### Funciones antrenamiento y evaluacion


In [7]:
# Definimos la funcion de entrenamiento
def train_global(model, train_loader, optimizer, criterion,epoch=None,writer=None):
    model.train()
    total_loss = 0
    total_metric = 0
    normalized_error = NormalizedError()

    for data in train_loader:
        optimizer.zero_grad()
        output = model(data[0])
        target = data[1]
        loss = criterion(output, target) 
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Calcular la métrica
        metric = normalized_error(output, target)
        total_metric += metric.item()

    avg_loss = total_loss / len(train_loader)
    avg_metric = total_metric / len(train_loader)

    writer.add_scalar('Train/Training Loss', avg_loss, epoch)
    writer.add_scalar('Train/Training Metric', avg_metric, epoch)
    
    return avg_loss, avg_metric


# Definimos la funcion de evaluacion
def evaluate_global(model, data_loader, criterion,epoch=None,writer=None,test=False):
    model.eval()
    total_loss = 0
    total_metric = 0
    normalized_error = NormalizedError()

    with torch.no_grad():
        for data in data_loader:
            output = model(data[0])
            target = data[1]
            loss = criterion(output, target)  # target should contain true values for nodes with missing features
            total_loss += loss.item()
            # Calcular la métrica
            metric = normalized_error(output, target)
            total_metric += metric.item()

    avg_loss = total_loss / len(data_loader)
    avg_metric = total_metric / len(data_loader)

    if epoch != None:
        writer.add_scalar('Val/Validation Loss', avg_loss, epoch)
        writer.add_scalar('Val/Validation Metric', avg_metric, epoch)
    
    return avg_loss, avg_metric


### GNN

In [9]:

writer_gnn = SummaryWriter('resultados/gnn_global_'+red)

# Hiperparametros
num_nodes = len(net.bus)
num_gens = len(net.gen) +len(net.ext_grid)
dim = [4,128,64,1]
num_layers = 3
K = [5,5]
num_epochs = 200

if red == "118":
    lr = 0.0001
else:
    lr = 0.0002

# Definicion Modelo, Optimizer y Loss
model_gnn = GNN_global(dim, edge_index,edge_weights, num_layers, K, num_nodes, num_gens,batch_norm=False).to(device)
optimizer = torch.optim.Adam(model_gnn.parameters(), lr=lr)
criterion = nn.MSELoss()

# Entrenamiento
for epoch in range(num_epochs):
    train_loss, train_metric = train_global(model_gnn, train_loader_global, optimizer, criterion,epoch,writer_gnn)
    val_loss, val_metric = evaluate_global(model_gnn, val_loader_global, criterion, epoch,writer_gnn)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Metric: {train_metric:.4f},  Val Loss: {val_loss:.4f}, Val Metric: {val_metric:.4f}")

# Evaluacion
test_loss, test_metric = evaluate_global(model_gnn, test_loader_global, criterion,None,writer_gnn)
print(f"Test Loss: {test_loss:.5f}, Test Metric: {test_metric:.5f}")

Epoch 1/200, Train Loss: 3934.6426, Train Metric: 0.4707,  Val Loss: 40.1176, Val Metric: 0.2006
Epoch 2/200, Train Loss: 11.7675, Train Metric: 0.1383,  Val Loss: 6.1383, Val Metric: 0.1171
Epoch 3/200, Train Loss: 6.1627, Train Metric: 0.1171,  Val Loss: 5.5537, Val Metric: 0.1143
Epoch 4/200, Train Loss: 5.6772, Train Metric: 0.1150,  Val Loss: 5.1494, Val Metric: 0.1124
Epoch 5/200, Train Loss: 5.2689, Train Metric: 0.1131,  Val Loss: 4.7872, Val Metric: 0.1106
Epoch 6/200, Train Loss: 4.8965, Train Metric: 0.1114,  Val Loss: 4.4469, Val Metric: 0.1090
Epoch 7/200, Train Loss: 4.5301, Train Metric: 0.1095,  Val Loss: 4.1136, Val Metric: 0.1073
Epoch 8/200, Train Loss: 4.1523, Train Metric: 0.1074,  Val Loss: 3.7601, Val Metric: 0.1052
Epoch 9/200, Train Loss: 3.7689, Train Metric: 0.1052,  Val Loss: 3.4203, Val Metric: 0.1032
Epoch 10/200, Train Loss: 3.4010, Train Metric: 0.1029,  Val Loss: 3.0966, Val Metric: 0.1011
Epoch 11/200, Train Loss: 3.0367, Train Metric: 0.1004,  Val Los

### FCNN

In [41]:
writer_fcnn = SummaryWriter('resultados/fcnn_global_'+red)

# Hiperparametros
num_nodes = len(net.bus)
num_gens = len(net.gen) +len(net.ext_grid)
dim = [4*num_nodes,128,64,num_gens]
num_layers = 3
num_epochs = 200
if red == "118":
    lr = 0.0003
else:
    lr = 0.00005

# Definicion Modelo, Optimizer y Loss
model_fcnn = FCNN_global(dim, num_layers).to(device)
optimizer = torch.optim.Adam(model_fcnn.parameters(), lr=lr)
criterion = nn.MSELoss()  # Change the loss function as needed

# Entrenamiento
for epoch in range(num_epochs):
    train_loss, train_metric = train_global(model_fcnn, train_loader_global, optimizer, criterion,epoch,writer_fcnn)
    val_loss, val_metric = evaluate_global(model_fcnn, val_loader_global, criterion, epoch,writer_fcnn)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Metric: {train_metric:.4f},  Val Loss: {val_loss:.4f}, Val Metric: {val_metric:.4f}")

# Evaluacion
test_loss, test_metric = evaluate_global(model_fcnn, test_loader_global, criterion,None,writer_fcnn)
print(f"Test Loss: {test_loss:.5f}, Test Metric: {test_metric:.5f}")

Epoch 1/200, Train Loss: 14610.9127, Train Metric: 0.8422,  Val Loss: 1580.5119, Val Metric: 0.5015
Epoch 2/200, Train Loss: 218.5167, Train Metric: 0.2368,  Val Loss: 4.1995, Val Metric: 0.1128
Epoch 3/200, Train Loss: 2.5504, Train Metric: 0.0983,  Val Loss: 2.1716, Val Metric: 0.0945
Epoch 4/200, Train Loss: 2.1933, Train Metric: 0.0947,  Val Loss: 2.1609, Val Metric: 0.0944
Epoch 5/200, Train Loss: 2.1895, Train Metric: 0.0946,  Val Loss: 2.1546, Val Metric: 0.0943
Epoch 6/200, Train Loss: 2.1856, Train Metric: 0.0946,  Val Loss: 2.1485, Val Metric: 0.0943
Epoch 7/200, Train Loss: 2.1811, Train Metric: 0.0946,  Val Loss: 2.1436, Val Metric: 0.0942
Epoch 8/200, Train Loss: 2.1763, Train Metric: 0.0945,  Val Loss: 2.1391, Val Metric: 0.0942
Epoch 9/200, Train Loss: 2.1713, Train Metric: 0.0945,  Val Loss: 2.1346, Val Metric: 0.0942
Epoch 10/200, Train Loss: 2.1661, Train Metric: 0.0944,  Val Loss: 2.1301, Val Metric: 0.0941
Epoch 11/200, Train Loss: 2.1608, Train Metric: 0.0944,  Val

In [ ]:

total_params = sum(p.numel() for p in model_gnn.parameters() if p.requires_grad)
total_params

438397

In [11]:
total_params = sum(p.numel() for p in model_fcnn.parameters() if p.requires_grad)
total_params

24069